## 1. Import Libraries

### Purpose:
Import essential Python libraries for data manipulation, numerical operations, and file management.

### Libraries Used:
- **pandas**: DataFrame operations and feature creation
- **numpy**: Mathematical operations and handling special values (inf, NaN)
- **os**: File path management for saving outputs

In [1]:
import pandas as pd
import numpy as np
import os

## 2. Load Cleaned Data

### Purpose:
Load the cleaned customer churn dataset that was prepared in the data cleaning notebook.

### What to Expect:
- Clean data with zero missing values
- Standardized categorical variables
- Validated numeric ranges
- 21 original features ready for enhancement

In [2]:
# Load the cleaned customer churn dataset
input_file = '../data/customer_churn_cleaned.csv'
df = pd.read_csv(input_file)

print(f"✓ Loaded {len(df)} records from: {input_file}")
print(f"\nDataset shape: {df.shape}")
print(f"Initial features: {len(df.columns)}")
print(f"\nColumns: {list(df.columns)}")

# Display first few rows
print("\n📄 Sample Data (first 5 rows):")
df.head()

✓ Loaded 100 records from: ../data/customer_churn_cleaned.csv

Dataset shape: (100, 21)
Initial features: 21

Columns: ['customer_id', 'age', 'gender', 'tenure_months', 'subscription_plan', 'monthly_charges', 'total_charges', 'contract_type', 'payment_method', 'login_frequency_monthly', 'features_used', 'data_consumption_gb', 'engagement_score', 'days_since_last_activity', 'billing_issues_count', 'plan_changes', 'support_tickets', 'avg_resolution_hours', 'satisfaction_score', 'nps_score', 'churned']

📄 Sample Data (first 5 rows):


,customer_id,age,gender,tenure_months,subscription_plan,monthly_charges,total_charges,contract_type,payment_method,login_frequency_monthly,...,data_consumption_gb,engagement_score,days_since_last_activity,billing_issues_count,plan_changes,support_tickets,avg_resolution_hours,satisfaction_score,nps_score,churned
0,C001,34.0,Male,24,Premium,79.99,1919.76,Annual,Credit Card,22.0,...,45.2,100.0,2,0,1,1,4.5,4.2,8,0
1,C002,45.0,Female,6,Basic,29.99,179.94,Monthly,Bank Transfer,5.0,...,8.3,25.0,15,2,0,4,48.2,2.1,3,1
2,C003,28.0,Male,36,Premium,79.99,2879.64,Annual,Credit Card,28.0,...,62.1,92.0,1,0,2,0,0.0,4.8,9,0
3,C004,52.0,Female,3,Basic,29.99,89.97,Monthly,Credit Card,3.0,...,4.2,18.0,22,1,0,3,36.5,2.5,2,1
4,C005,31.0,Male,1,Standard,49.99,899.82,Monthly,PayPal,15.0,...,28.4,65.0,5,0,0,2,12.3,3.8,7,0


### Interpretation:
- **Initial features**: Starting with 21 clean features from previous step
- **Clean state**: No missing values or data quality issues
- **Ready for engineering**: Data is prepared for transformation

## 3. Category 1: Customer Value Features

### Purpose:
Create features that capture customer monetary value and revenue patterns.

### Features to Create:
1. **monthly_value_ratio** (ARPU): Average Revenue Per User = total_charges / tenure_months
2. **charge_per_feature**: Cost efficiency = monthly_charges / features_used
3. **customer_lifetime_value**: Total revenue from non-churned customers
4. **value_tier**: Customer segmentation (Low, Medium, High, Premium)

### Why These Matter:
- **ARPU**: Identifies high-value customers worth retaining
- **Charge per feature**: Detects over/under-monetization
- **CLV**: Proxy for customer worth to business
- **Value tier**: Enables segment-specific retention strategies

### Business Insights:
- High ARPU customers with low engagement may churn
- Customers paying a lot per feature may feel overcharged
- Value tier helps prioritize retention efforts

In [3]:
print("="*70)
print("CATEGORY 1: CUSTOMER VALUE FEATURES")
print("="*70)

# Monthly value ratio (ARPU - Average Revenue Per User)
df['monthly_value_ratio'] = df['total_charges'] / df['tenure_months']
df['monthly_value_ratio'] = df['monthly_value_ratio'].replace([np.inf, -np.inf], 0)
print("\n✓ Created 'monthly_value_ratio' (ARPU)")
print(f"   Mean ARPU: ${df['monthly_value_ratio'].mean():.2f}")
print(f"   Median ARPU: ${df['monthly_value_ratio'].median():.2f}")

# Charge efficiency (cost per feature)
df['charge_per_feature'] = df['monthly_charges'] / df['features_used']
df['charge_per_feature'] = df['charge_per_feature'].replace([np.inf, -np.inf], 0)
print("\n✓ Created 'charge_per_feature'")
print(f"   Mean: ${df['charge_per_feature'].mean():.2f}")
print(f"   Range: ${df['charge_per_feature'].min():.2f} - ${df['charge_per_feature'].max():.2f}")

# Lifetime value proxy (only for non-churned customers)
df['customer_lifetime_value'] = df.apply(
    lambda row: row['total_charges'] if row['churned'] == 0 else 0, axis=1
)
print("\n✓ Created 'customer_lifetime_value'")
print(f"   Total CLV (non-churned): ${df['customer_lifetime_value'].sum():,.2f}")
print(f"   Average CLV: ${df['customer_lifetime_value'].mean():.2f}")

# Value tier based on total charges
df['value_tier'] = pd.cut(
    df['total_charges'],
    bins=[0, 500, 1500, 3000, np.inf],
    labels=['Low', 'Medium', 'High', 'Premium']
)
print("\n✓ Created 'value_tier'")
print(f"   Distribution:\n{df['value_tier'].value_counts().sort_index()}")

print(f"\n📊 Value Features Summary:")
print(f"   Features created: 4")
print(f"   New columns: {len(df.columns)} total")

CATEGORY 1: CUSTOMER VALUE FEATURES

✓ Created 'monthly_value_ratio' (ARPU)
   Mean ARPU: $111.01
   Median ARPU: $49.99

✓ Created 'charge_per_feature'
   Mean: $15.38
   Range: $0.00 - $49.99

✓ Created 'customer_lifetime_value'
   Total CLV (non-churned): $108,583.86
   Average CLV: $1085.84

✓ Created 'value_tier'
   Distribution:
value_tier
Low        33
Medium     35
High       21
Premium    11
Name: count, dtype: int64

📊 Value Features Summary:
   Features created: 4
   New columns: 25 total


### Interpretation:
- **ARPU metrics**: Shows average monthly revenue per customer
- **Charge per feature**: Identifies pricing efficiency
- **CLV**: Total value of retained customers
- **Value tier distribution**: Customer segmentation by spending

## 4. Category 2: Engagement Features

### Purpose:
Create features capturing customer engagement, activity, and usage patterns.

### Features to Create:
1. **engagement_velocity**: Engagement rate over time = engagement_score / tenure_months
2. **login_intensity**: Average daily logins = login_frequency_monthly / 30
3. **data_per_login**: Data usage efficiency = data_consumption_gb / login_frequency_monthly
4. **activity_recency_category**: Active, Moderate, At_Risk, Dormant
5. **features_utilization_rate**: Percentage of features used (out of 10)
6. **data_per_tenure**: Data consumption normalized by tenure

### Why These Matter:
- **Engagement velocity**: Identifies declining engagement trends
- **Login intensity**: Measures daily platform stickiness
- **Data per login**: Heavy users vs casual browsers
- **Activity recency**: Early warning signal for churn
- **Feature utilization**: Product adoption and value realization

### Churn Indicators:
- Decreasing engagement velocity → Losing interest
- Low login intensity → Not forming habit
- High days since last activity → Disengagement risk

In [4]:
print("\n" + "="*70)
print("CATEGORY 2: ENGAGEMENT FEATURES")
print("="*70)

# Engagement velocity (engagement per month)
df['engagement_velocity'] = df['engagement_score'] / df['tenure_months']
df['engagement_velocity'] = df['engagement_velocity'].replace([np.inf, -np.inf], 0)
print("\n✓ Created 'engagement_velocity'")
print(f"   Mean: {df['engagement_velocity'].mean():.2f}")
print(f"   High velocity (>10): {(df['engagement_velocity'] > 10).sum()} customers")

# Login intensity (average daily logins)
df['login_intensity'] = df['login_frequency_monthly'] / 30
print("\n✓ Created 'login_intensity'")
print(f"   Mean daily logins: {df['login_intensity'].mean():.2f}")
print(f"   Daily active users (>0.5): {(df['login_intensity'] > 0.5).sum()} customers")

# Data per login
df['data_per_login'] = df['data_consumption_gb'] / df['login_frequency_monthly']
df['data_per_login'] = df['data_per_login'].replace([np.inf, -np.inf], 0)
print("\n✓ Created 'data_per_login'")
print(f"   Mean GB per login: {df['data_per_login'].mean():.2f}")

# Activity recency category
df['activity_recency_category'] = pd.cut(
    df['days_since_last_activity'],
    bins=[-1, 7, 14, 30, np.inf],
    labels=['Active', 'Moderate', 'At_Risk', 'Dormant']
)
print("\n✓ Created 'activity_recency_category'")
print(f"   Distribution:\n{df['activity_recency_category'].value_counts()}")

# Features utilization rate
df['features_utilization_rate'] = df['features_used'] / 10  # Assuming max 10 features
print("\n✓ Created 'features_utilization_rate'")
print(f"   Mean utilization: {df['features_utilization_rate'].mean():.1%}")
print(f"   Full utilization (100%): {(df['features_utilization_rate'] == 1.0).sum()} customers")

# Data consumption per tenure
df['data_per_tenure'] = df['data_consumption_gb'] / df['tenure_months']
print("\n✓ Created 'data_per_tenure'")
print(f"   Mean GB per month of tenure: {df['data_per_tenure'].mean():.2f}")

print(f"\n📊 Engagement Features Summary:")
print(f"   Features created: 6")
print(f"   New columns: {len(df.columns)} total")


CATEGORY 2: ENGAGEMENT FEATURES

✓ Created 'engagement_velocity'
   Mean: 5.55
   High velocity (>10): 5 customers

✓ Created 'login_intensity'
   Mean daily logins: 0.44
   Daily active users (>0.5): 43 customers

✓ Created 'data_per_login'
   Mean GB per login: 2.08

✓ Created 'activity_recency_category'
   Distribution:
activity_recency_category
Active      52
At_Risk     38
Moderate    10
Dormant      0
Name: count, dtype: int64

✓ Created 'features_utilization_rate'
   Mean utilization: 48.8%
   Full utilization (100%): 4 customers

✓ Created 'data_per_tenure'
   Mean GB per month of tenure: 2.42

📊 Engagement Features Summary:
   Features created: 6
   New columns: 31 total


### Interpretation:
- **Velocity metrics**: Shows engagement trends over time
- **Intensity measures**: Daily and per-login usage patterns
- **Recency categories**: Customer activity segmentation
- **Utilization rates**: Product adoption levels

## 5. Category 3: Support Risk Features

### Purpose:
Create features identifying customer satisfaction issues and support burden.

### Features to Create:
1. **support_rate_annual**: Annualized support ticket rate
2. **resolution_burden**: Total hours spent resolving issues
3. **satisfaction_gap**: Distance from perfect satisfaction (5)
4. **billing_risk_flag**: Binary indicator of billing issues
5. **complaint_ratio**: Combined billing + plan change rate
6. **support_satisfaction_ratio**: Satisfaction per support interaction
7. **nps_category**: Detractor, Passive, or Promoter

### Why These Matter:
- **Support rate**: High ticket volume indicates problems
- **Resolution burden**: Time-intensive customers may churn
- **Satisfaction gap**: Quantifies dissatisfaction level
- **Billing risk**: Payment issues predict churn
- **NPS category**: Net Promoter Score groups (industry standard)

### Risk Indicators:
- High support rate + low satisfaction = High churn risk
- Billing issues are strong churn predictors
- Detractors (NPS 0-6) are very likely to churn

In [5]:
print("\n" + "="*70)
print("CATEGORY 3: SUPPORT RISK FEATURES")
print("="*70)

# Support rate (annualized)
df['support_rate_annual'] = (df['support_tickets'] / df['tenure_months']) * 12
print("\n✓ Created 'support_rate_annual'")
print(f"   Mean annual tickets: {df['support_rate_annual'].mean():.2f}")
print(f"   High support (>6 tickets/year): {(df['support_rate_annual'] > 6).sum()} customers")

# Resolution burden (total resolution time)
df['resolution_burden'] = df['avg_resolution_hours'] * df['support_tickets']
print("\n✓ Created 'resolution_burden'")
print(f"   Mean total resolution hours: {df['resolution_burden'].mean():.2f}")
print(f"   High burden (>100 hours): {(df['resolution_burden'] > 100).sum()} customers")

# Satisfaction gap (distance from perfect score)
df['satisfaction_gap'] = 5 - df['satisfaction_score']
print("\n✓ Created 'satisfaction_gap'")
print(f"   Mean gap: {df['satisfaction_gap'].mean():.2f}")
print(f"   Perfect satisfaction (gap=0): {(df['satisfaction_gap'] == 0).sum()} customers")

# Billing risk flag
df['billing_risk_flag'] = (df['billing_issues_count'] > 0).astype(int)
print("\n✓ Created 'billing_risk_flag'")
print(f"   Customers with billing issues: {df['billing_risk_flag'].sum()} ({df['billing_risk_flag'].mean():.1%})")

# Complaint ratio
df['complaint_ratio'] = (
    (df['billing_issues_count'] + df['plan_changes']) / df['tenure_months']
)
df['complaint_ratio'] = df['complaint_ratio'].replace([np.inf, -np.inf], 0)
print("\n✓ Created 'complaint_ratio'")
print(f"   Mean monthly complaint rate: {df['complaint_ratio'].mean():.3f}")

# Support satisfaction ratio
df['support_satisfaction_ratio'] = (
    df['satisfaction_score'] / (df['support_tickets'] + 1)
)
print("\n✓ Created 'support_satisfaction_ratio'")
print(f"   Mean: {df['support_satisfaction_ratio'].mean():.2f}")

# NPS category
df['nps_category'] = pd.cut(
    df['nps_score'],
    bins=[-1, 6, 8, 10],
    labels=['Detractor', 'Passive', 'Promoter']
)
print("\n✓ Created 'nps_category'")
print(f"   Distribution:\n{df['nps_category'].value_counts()}")
print(f"   Net Promoter Score: {((df['nps_category'] == 'Promoter').sum() - (df['nps_category'] == 'Detractor').sum()) / len(df) * 100:.1f}")

print(f"\n📊 Support Risk Features Summary:")
print(f"   Features created: 7")
print(f"   New columns: {len(df.columns)} total")


CATEGORY 3: SUPPORT RISK FEATURES

✓ Created 'support_rate_annual'
   Mean annual tickets: 6.26
   High support (>6 tickets/year): 34 customers

✓ Created 'resolution_burden'
   Mean total resolution hours: 78.52
   High burden (>100 hours): 32 customers

✓ Created 'satisfaction_gap'
   Mean gap: 1.52
   Perfect satisfaction (gap=0): 1 customers

✓ Created 'billing_risk_flag'
   Customers with billing issues: 46 (46.0%)

✓ Created 'complaint_ratio'
   Mean monthly complaint rate: 0.194

✓ Created 'support_satisfaction_ratio'
   Mean: 1.66

✓ Created 'nps_category'
   Distribution:
nps_category
Detractor    50
Passive      32
Promoter     18
Name: count, dtype: int64
   Net Promoter Score: -32.0

📊 Support Risk Features Summary:
   Features created: 7
   New columns: 38 total


### Interpretation:
- **Support metrics**: Annual ticket volume and resolution time
- **Satisfaction measures**: Gaps and risk flags
- **NPS distribution**: Customer advocacy levels
- **Risk indicators**: Billing issues and complaint rates

## 6. Category 4: Interaction Features

### Purpose:
Create features capturing mismatches and interactions between different customer attributes.

### Features to Create:
1. **plan_tenure_mismatch**: Premium users who are new or Basic users who are veteran
2. **usage_plan_mismatch**: High usage on Basic or low usage on Premium
3. **payment_stability**: Inverse of billing issues (stability score)
4. **nps_satisfaction_alignment**: Consistency between NPS and satisfaction
5. **contract_value_risk**: Value-contract type mismatch

### Why These Matter:
- **Plan-tenure mismatch**: New Premium customers may have buyer's remorse
- **Usage-plan mismatch**: Wrong plan → dissatisfaction → churn
- **Payment stability**: Payment issues are strong churn predictors
- **NPS-satisfaction alignment**: Inconsistency indicates issues
- **Contract-value risk**: Wrong contract type for value level

### Business Logic:
- Premium customers should be mature (not new)
- High data users should be on Premium plans
- Low data users on Premium are wasting money (churn risk)
- Misalignments create dissatisfaction

In [6]:
print("\n" + "="*70)
print("CATEGORY 4: INTERACTION FEATURES")
print("="*70)

# Plan tenure fit
def plan_tenure_fit(row):
    if row['subscription_plan'] == 'Premium' and row['tenure_months'] < 6:
        return 1  # Risky: Premium but new
    elif row['subscription_plan'] == 'Basic' and row['tenure_months'] > 36:
        return 1  # Risky: Basic but long tenure (undermonetized)
    else:
        return 0

df['plan_tenure_mismatch'] = df.apply(plan_tenure_fit, axis=1)
print("\n✓ Created 'plan_tenure_mismatch'")
print(f"   Mismatched customers: {df['plan_tenure_mismatch'].sum()} ({df['plan_tenure_mismatch'].mean():.1%})")
print(f"   - New Premium users: {((df['subscription_plan'] == 'Premium') & (df['tenure_months'] < 6)).sum()}")
print(f"   - Veteran Basic users: {((df['subscription_plan'] == 'Basic') & (df['tenure_months'] > 36)).sum()}")

# Usage plan mismatch
def usage_plan_mismatch(row):
    if row['subscription_plan'] == 'Basic' and row['data_consumption_gb'] > 40:
        return 1  # High usage on basic plan
    elif row['subscription_plan'] == 'Premium' and row['data_consumption_gb'] < 20:
        return 1  # Low usage on premium plan
    else:
        return 0

df['usage_plan_mismatch'] = df.apply(usage_plan_mismatch, axis=1)
print("\n✓ Created 'usage_plan_mismatch'")
print(f"   Mismatched customers: {df['usage_plan_mismatch'].sum()} ({df['usage_plan_mismatch'].mean():.1%})")
print(f"   - High usage on Basic: {((df['subscription_plan'] == 'Basic') & (df['data_consumption_gb'] > 40)).sum()}")
print(f"   - Low usage on Premium: {((df['subscription_plan'] == 'Premium') & (df['data_consumption_gb'] < 20)).sum()}")

# Payment stability
df['payment_stability'] = 1 / (df['billing_issues_count'] + 1)
print("\n✓ Created 'payment_stability'")
print(f"   Mean stability: {df['payment_stability'].mean():.2f}")
print(f"   Perfect stability (1.0): {(df['payment_stability'] == 1.0).sum()} customers")

# NPS satisfaction alignment
df['nps_satisfaction_alignment'] = abs((df['nps_score'] / 2) - df['satisfaction_score'])
print("\n✓ Created 'nps_satisfaction_alignment'")
print(f"   Mean misalignment: {df['nps_satisfaction_alignment'].mean():.2f}")
print(f"   Perfect alignment (0.0): {(df['nps_satisfaction_alignment'] == 0.0).sum()} customers")

# Contract value match
def contract_value_match(row):
    if row['contract_type'] == 'Annual' and row['monthly_charges'] < 40:
        return 1  # Low value annual contract
    elif row['contract_type'] == 'Monthly' and row['monthly_charges'] > 70:
        return 1  # High value monthly (retention risk)
    else:
        return 0

df['contract_value_risk'] = df.apply(contract_value_match, axis=1)
print("\n✓ Created 'contract_value_risk'")
print(f"   At-risk customers: {df['contract_value_risk'].sum()} ({df['contract_value_risk'].mean():.1%})")
print(f"   - Low-value annual: {((df['contract_type'] == 'Annual') & (df['monthly_charges'] < 40)).sum()}")
print(f"   - High-value monthly: {((df['contract_type'] == 'Monthly') & (df['monthly_charges'] > 70)).sum()}")

print(f"\n📊 Interaction Features Summary:")
print(f"   Features created: 5")
print(f"   New columns: {len(df.columns)} total")


CATEGORY 4: INTERACTION FEATURES

✓ Created 'plan_tenure_mismatch'
   Mismatched customers: 1 (1.0%)
   - New Premium users: 1
   - Veteran Basic users: 0

✓ Created 'usage_plan_mismatch'
   Mismatched customers: 4 (4.0%)
   - High usage on Basic: 0
   - Low usage on Premium: 4

✓ Created 'payment_stability'
   Mean stability: 0.72
   Perfect stability (1.0): 54 customers

✓ Created 'nps_satisfaction_alignment'
   Mean misalignment: 0.68
   Perfect alignment (0.0): 4 customers

✓ Created 'contract_value_risk'
   At-risk customers: 9 (9.0%)
   - Low-value annual: 0
   - High-value monthly: 9

📊 Interaction Features Summary:
   Features created: 5
   New columns: 43 total


### Interpretation:
- **Mismatch detection**: Identifies customers on wrong plans
- **Stability scores**: Payment reliability indicators
- **Alignment measures**: Consistency checks between metrics
- **Risk flags**: Specific at-risk customer segments

## 7. Category 5: Tenure & Lifecycle Features

### Purpose:
Create features capturing customer lifecycle stage and tenure-based patterns.

### Features to Create:
1. **lifecycle_stage**: New, Trial_End, Early, Mature, Veteran
2. **contract_tenure_ratio**: How many contract periods completed
3. **tenure_category**: Simple tenure bins (0-1yr, 1-2yr, 2-3yr, 3yr+)
4. **engagement_growth_rate**: Engagement per month of tenure
5. **tenure_stability**: Tenure normalized by customer age

### Why These Matter:
- **Lifecycle stage**: Different churn risks at different stages
- **Contract tenure ratio**: Risk highest at contract renewal
- **Tenure category**: Enables stage-specific strategies
- **Engagement growth**: Positive trend = lower churn risk
- **Tenure stability**: Long tenure relative to age = loyal customer

### Critical Stages:
- **Trial End (6-12 months)**: High churn risk if value not proven
- **Contract renewal**: Annual contracts are vulnerable at 12-month mark
- **New customers (<6 months)**: Onboarding quality critical

In [7]:
print("\n" + "="*70)
print("CATEGORY 5: TENURE & LIFECYCLE FEATURES")
print("="*70)

# Lifecycle stage
df['lifecycle_stage'] = pd.cut(
    df['tenure_months'],
    bins=[0, 6, 12, 24, 48, np.inf],
    labels=['New', 'Trial_End', 'Early', 'Mature', 'Veteran']
)
print("\n✓ Created 'lifecycle_stage'")
print(f"   Distribution:\n{df['lifecycle_stage'].value_counts().sort_index()}")

# Contract tenure ratio
contract_months = {'Monthly': 1, 'Annual': 12}
df['contract_tenure_ratio'] = df.apply(
    lambda row: row['tenure_months'] / contract_months.get(row['contract_type'], 1),
    axis=1
)
print("\n✓ Created 'contract_tenure_ratio'")
print(f"   Mean contract periods completed: {df['contract_tenure_ratio'].mean():.2f}")
print(f"   Multiple renewals (>2 periods): {(df['contract_tenure_ratio'] > 2).sum()} customers")

# Tenure category (simple bins)
df['tenure_category'] = pd.cut(
    df['tenure_months'],
    bins=[0, 12, 24, 36, np.inf],
    labels=['0-1yr', '1-2yr', '2-3yr', '3yr+']
)
print("\n✓ Created 'tenure_category'")
print(f"   Distribution:\n{df['tenure_category'].value_counts().sort_index()}")

# Growth rate (engagement growth over tenure)
df['engagement_growth_rate'] = df['engagement_score'] / (df['tenure_months'] + 1)
print("\n✓ Created 'engagement_growth_rate'")
print(f"   Mean growth rate: {df['engagement_growth_rate'].mean():.2f}")
print(f"   High growth (>5): {(df['engagement_growth_rate'] > 5).sum()} customers")

# Tenure stability score (tenure normalized by age)
df['tenure_stability'] = df['tenure_months'] / (df['age'] * 12)
print("\n✓ Created 'tenure_stability'")
print(f"   Mean stability: {df['tenure_stability'].mean():.3f}")
print(f"   High stability (>0.1): {(df['tenure_stability'] > 0.1).sum()} customers")

print(f"\n📊 Tenure & Lifecycle Features Summary:")
print(f"   Features created: 5")
print(f"   New columns: {len(df.columns)} total")


CATEGORY 5: TENURE & LIFECYCLE FEATURES

✓ Created 'lifecycle_stage'
   Distribution:
lifecycle_stage
New          36
Trial_End    14
Early        15
Mature       29
Veteran       6
Name: count, dtype: int64

✓ Created 'contract_tenure_ratio'
   Mean contract periods completed: 4.50
   Multiple renewals (>2 periods): 80 customers

✓ Created 'tenure_category'
   Distribution:
tenure_category
0-1yr    50
1-2yr    15
2-3yr    18
3yr+     17
Name: count, dtype: int64

✓ Created 'engagement_growth_rate'
   Mean growth rate: 4.25
   High growth (>5): 12 customers

✓ Created 'tenure_stability'
   Mean stability: 0.039
   High stability (>0.1): 6 customers

📊 Tenure & Lifecycle Features Summary:
   Features created: 5
   New columns: 48 total


### Interpretation:
- **Lifecycle distribution**: Customer base composition by stage
- **Contract completion**: How many renewal cycles completed
- **Tenure segmentation**: Simple categorization for analysis
- **Growth metrics**: Engagement trends over time
- **Stability measures**: Long-term loyalty indicators

## 8. Feature Validation

### Purpose:
Validate all engineered features for data quality issues and assess predictive power.

### Validation Checks:
1. ✓ No infinite values (replace with 0)
2. ✓ No NaN values (fill appropriately)
3. ✓ Calculate feature correlations with churn target
4. ✓ Identify top predictive features

### Why Validation Matters:
- **Infinite values**: Can crash model training
- **NaN values**: Need to be handled before modeling
- **Correlation analysis**: Shows which features are most predictive
- **Quality assurance**: Ensures features are ready for ML

### What to Look For:
- Strong correlations (|r| > 0.3) indicate good features
- Features with zero variance should be removed
- Categorical features properly encoded

In [8]:
print("\n" + "="*70)
print("FEATURE VALIDATION")
print("="*70)

# Check for infinite values in numeric columns
inf_cols = []
for col in df.select_dtypes(include=[np.number]).columns:
    if np.isinf(df[col]).any():
        inf_cols.append(col)

if inf_cols:
    print(f"\n⚠️  Found infinite values in: {inf_cols}")
    # Replace inf with 0
    df = df.replace([np.inf, -np.inf], 0)
    print("   ✓ Replaced infinite values with 0")
else:
    print("\n✓ No infinite values found")

# Check for NaN values
nan_count = df.isnull().sum().sum()
if nan_count > 0:
    print(f"\n⚠️  Found {nan_count} NaN values")
    
    # Fill NaN for numeric columns only
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna(0)
    
    # For categorical columns, fill with mode or add 'Unknown' category
    categorical_cols = df.select_dtypes(include=['category', 'object']).columns
    for col in categorical_cols:
        if df[col].isnull().any():
            if col not in ['customer_id', 'churned']:
                mode_val = df[col].mode()
                if len(mode_val) > 0:
                    df[col] = df[col].fillna(mode_val[0])
                else:
                    if hasattr(df[col], 'cat'):
                        df[col] = df[col].cat.add_categories(['Unknown']).fillna('Unknown')
    
    print("   ✓ Filled NaN values (0 for numeric, mode for categorical)")
else:
    print("\n✓ No NaN values found")

# Check feature correlations with target
print("\n" + "="*70)
print("TOP 15 FEATURES CORRELATED WITH CHURN")
print("="*70)

numeric_df = df.select_dtypes(include=[np.number])
if 'churned' in numeric_df.columns:
    correlations = numeric_df.corr()['churned'].sort_values(ascending=False)
    top_features = correlations[1:16]  # Exclude 'churned' itself
    
    print("\nFeature Name                          Correlation")
    print("-" * 70)
    for i, (feature, corr) in enumerate(top_features.items(), 1):
        bar = "█" * int(abs(corr) * 50)
        sign = "+" if corr > 0 else "-"
        print(f"{i:2d}. {feature:35s} {sign}{abs(corr):.4f}  {bar}")
    
    # Categorize by strength
    strong = correlations[(correlations.abs() > 0.3) & (correlations.abs() < 1.0)]
    moderate = correlations[(correlations.abs() > 0.15) & (correlations.abs() <= 0.3)]
    
    print(f"\n📊 Correlation Strength Summary:")
    print(f"   Strong correlations (|r| > 0.3): {len(strong)}")
    print(f"   Moderate correlations (0.15 < |r| ≤ 0.3): {len(moderate)}")
    print(f"   Weak correlations (|r| ≤ 0.15): {len(correlations) - len(strong) - len(moderate) - 1}")
else:
    print("   ⚠️  'churned' column not found in numeric columns")

print("\n✅ Feature validation complete!")


FEATURE VALIDATION

✓ No infinite values found

⚠️  Found 26 NaN values
   ✓ Filled NaN values (0 for numeric, mode for categorical)

TOP 15 FEATURES CORRELATED WITH CHURN

Feature Name                          Correlation
----------------------------------------------------------------------
 1. billing_risk_flag                   +0.9604  ████████████████████████████████████████████████
 2. days_since_last_activity            +0.9194  █████████████████████████████████████████████
 3. avg_resolution_hours                +0.9182  █████████████████████████████████████████████
 4. billing_issues_count                +0.8926  ████████████████████████████████████████████
 5. support_tickets                     +0.8686  ███████████████████████████████████████████
 6. resolution_burden                   +0.8417  ██████████████████████████████████████████
 7. charge_per_feature                  +0.7806  ███████████████████████████████████████
 8. complaint_ratio                     +0.7772  

### Interpretation:
- **Validation results**: All quality checks passed
- **Correlation rankings**: Features sorted by predictive power
- **Strength indicators**: Visual bars show correlation magnitude
- **Feature quality**: Strong correlations indicate good engineering

## 9. Feature Summary & Documentation

### Purpose:
Generate comprehensive documentation of all features for reference and interpretability.

### Documentation Includes:
- Feature name and type (Original vs Engineered)
- Data type (int, float, category, object)
- Missing value count
- Unique value count
- Feature category

### Why Documentation Matters:
- **Interpretability**: Understand what each feature represents
- **Reproducibility**: Others can understand feature creation
- **Model explainability**: Connect features to business logic
- **Maintenance**: Easy to update and modify features

In [9]:
print("\n" + "="*70)
print("FEATURE SUMMARY & DOCUMENTATION")
print("="*70)

# Create feature catalog
features = []

# Original features list
original_features = [
    'customer_id', 'age', 'gender', 'tenure_months', 'subscription_plan',
    'monthly_charges', 'total_charges', 'contract_type', 'payment_method',
    'login_frequency_monthly', 'features_used', 'data_consumption_gb',
    'engagement_score', 'days_since_last_activity', 'billing_issues_count',
    'plan_changes', 'support_tickets', 'avg_resolution_hours',
    'satisfaction_score', 'nps_score', 'churned'
]

# Categorize features
for feature in df.columns:
    if feature in original_features:
        features.append({
            'feature': feature,
            'type': 'Original',
            'data_type': str(df[feature].dtype),
            'missing': df[feature].isnull().sum(),
            'unique': df[feature].nunique()
        })
    else:
        features.append({
            'feature': feature,
            'type': 'Engineered',
            'data_type': str(df[feature].dtype),
            'missing': df[feature].isnull().sum(),
            'unique': df[feature].nunique()
        })

feature_doc = pd.DataFrame(features)

# Display documentation
print("\n📋 Feature Catalog (first 20 features):")
display(feature_doc.head(20))

print("\n📊 Feature Summary:")
print(f"   Original features: {len([f for f in features if f['type'] == 'Original'])}")
print(f"   Engineered features: {len([f for f in features if f['type'] == 'Engineered'])}")
print(f"   Total features: {len(features)}")
print(f"   New features created: {len([f for f in features if f['type'] == 'Engineered'])}")

print("\n📈 Data Type Distribution:")
print(feature_doc['data_type'].value_counts())

print("\n✅ Feature quality:")
print(f"   Missing values: {feature_doc['missing'].sum()}")
print(f"   Features with zero variance: {len(feature_doc[feature_doc['unique'] == 1])}")

# Save documentation
doc_path = '../data/feature_documentation.csv'
feature_doc.to_csv(doc_path, index=False)
print(f"\n💾 Saved feature documentation to: {doc_path}")


FEATURE SUMMARY & DOCUMENTATION

📋 Feature Catalog (first 20 features):


,feature,type,data_type,missing,unique
0,customer_id,Original,object,0,100
1,age,Original,float64,0,41
2,gender,Original,object,0,2
3,tenure_months,Original,int64,0,31
4,subscription_plan,Original,object,0,3
5,monthly_charges,Original,float64,0,21
6,total_charges,Original,float64,0,36
7,contract_type,Original,object,0,2
8,payment_method,Original,object,0,3
9,login_frequency_monthly,Original,float64,0,26



📊 Feature Summary:
   Original features: 21
   Engineered features: 27
   Total features: 48
   New features created: 27

📈 Data Type Distribution:
data_type
float64     26
int64       12
object       5
category     5
Name: count, dtype: int64

✅ Feature quality:
   Missing values: 0
   Features with zero variance: 0

💾 Saved feature documentation to: ../data/feature_documentation.csv


### Interpretation:
- **Feature catalog**: Complete inventory of all features
- **Type distribution**: Original vs engineered breakdown
- **Quality metrics**: Missing values and variance
- **Documentation saved**: Reference file for future use

## 10. Save Featured Dataset

### Purpose:
Export the enhanced dataset with all engineered features for model training.

### Output Files:
1. **customer_churn_featured.csv**: Complete dataset with all features
2. **feature_documentation.csv**: Feature catalog and metadata

### What's Included:
- 21 original features (cleaned)
- 27+ engineered features (created)
- All features validated and quality-checked
- Ready for exploratory data analysis and model training

### Next Steps:
The featured dataset is now ready for:
- Exploratory data analysis (notebook 03)
- Model training and evaluation (notebook 04)
- Hyperparameter tuning (notebook 05)

In [10]:
# Save featured data
output_file = '../data/customer_churn_featured.csv'
df.to_csv(output_file, index=False)

print("="*70)
print("💾 SAVING FEATURED DATASET")
print("="*70)

print(f"\n✓ Saved featured data to: {output_file}")
print(f"   Records saved: {len(df):,}")
print(f"   Total features: {len(df.columns)}")
print(f"   File size: {os.path.getsize(output_file) / 1024:.2f} KB")

# Display sample of featured data
print("\n📄 Sample Featured Data (first 5 rows, selected columns):")
sample_cols = ['customer_id', 'churned', 'monthly_value_ratio', 'engagement_velocity', 
               'support_rate_annual', 'lifecycle_stage', 'value_tier']
display(df[sample_cols].head())

print("\n" + "="*70)
print("✅ FEATURE ENGINEERING COMPLETE")
print("="*70)
print(f"\n📊 Final Summary:")
print(f"   Initial features: 21 (from cleaned data)")
print(f"   Engineered features: {len(df.columns) - 21}")
print(f"   Total features: {len(df.columns)}")
print(f"   Records: {len(df):,}")
print(f"\n🎯 Ready for exploratory data analysis and model training!")
print("="*70)

💾 SAVING FEATURED DATASET

✓ Saved featured data to: ../data/customer_churn_featured.csv
   Records saved: 100
   Total features: 48
   File size: 33.29 KB

📄 Sample Featured Data (first 5 rows, selected columns):


,customer_id,churned,monthly_value_ratio,engagement_velocity,support_rate_annual,lifecycle_stage,value_tier
0,C001,0,79.99,4.166667,0.5,Early,High
1,C002,1,29.99,4.166667,8.0,New,Low
2,C003,0,79.99,2.555556,0.0,Mature,High
3,C004,1,29.99,6.000000,12.0,New,Low
4,C005,0,899.82,65.000000,24.0,New,Medium



✅ FEATURE ENGINEERING COMPLETE

📊 Final Summary:
   Initial features: 21 (from cleaned data)
   Engineered features: 27
   Total features: 48
   Records: 100

🎯 Ready for exploratory data analysis and model training!


### Interpretation:
- **Save successful**: Dataset written to file
- **Feature count**: Total and breakdown by category
- **Sample data**: Preview of engineered features
- **Ready for next stage**: Can proceed to EDA and modeling

## 11. Feature Engineering Summary

### What We Accomplished:

✅ **Customer Value Features (4)**: Revenue metrics, ARPU, CLV, value tiers  
✅ **Engagement Features (6)**: Activity patterns, login intensity, data usage  
✅ **Support Risk Features (7)**: Satisfaction gaps, support burden, NPS categories  
✅ **Interaction Features (5)**: Plan mismatches, payment stability, alignment checks  
✅ **Tenure & Lifecycle Features (5)**: Customer stage, contract completion, growth rates  
✅ **Validated All Features**: No infinite/NaN values, correlation analysis complete  
✅ **Documented Features**: Complete catalog with metadata  

### Feature Engineering Impact:

**Before**: 21 original features  
**After**: 48+ total features (27+ engineered)  
**Improvement**: 2.3x increase in feature space

### Top Predictive Features Created:
Based on correlation analysis, our engineered features likely include some of the strongest predictors of churn, particularly:
- Support and satisfaction metrics
- Engagement velocity and trends
- Plan-usage mismatches
- Lifecycle stage indicators

### Key Takeaways:

1. **Domain Knowledge**: Business understanding drove feature creation
2. **Interaction Effects**: Cross-feature patterns captured (plan × usage, NPS × satisfaction)
3. **Temporal Patterns**: Rates and ratios normalized by tenure
4. **Risk Indicators**: Binary flags for high-risk conditions
5. **Segmentation**: Categorical groupings for targeted strategies

### Files Generated:
- `customer_churn_featured.csv` - Enhanced dataset with all features
- `feature_documentation.csv` - Feature catalog and metadata

### Next Steps:
Proceed to **Exploratory Data Analysis** (notebook 03) to:
- Visualize feature distributions
- Analyze correlations with churn
- Identify patterns and insights
- Prepare for model training

### Best Practices Demonstrated:
✓ Handle division by zero (replace inf with 0)  
✓ Validate all features for quality issues  
✓ Document feature creation logic  
✓ Test correlations with target variable  
✓ Create both numeric and categorical features  
✓ Use domain knowledge to engineer meaningful features  